In [169]:
from google.colab import files
uploaded = files.upload()

In [0]:
import keras as ks
import pandas as pd
import numpy as np
import re

In [171]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
all = pd.concat([train, test], axis =0).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


Nameが「苗字, 敬称. 名前」となっているので苗字と敬称を新たな変数に。
Cabinに欠損が多いが欠損していないものは先頭のアルファベットを、欠損しているものは「U」を入れて新たな変数に。

In [172]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
    return kesson_table_ren_columns

kesson_table(all)

,欠損数,%
Age,263,20.091673
Cabin,1014,77.463713
Embarked,2,0.152788
Fare,1,0.076394
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [0]:
all["Surname"] = all["Name"].apply(lambda x:x.split(",")[0])
all["Title"] = all["Name"].apply(lambda x:re.split("[,.]", x)[1])

In [174]:
all.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Surname,Title
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Braund,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Cumings,Mrs
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Heikkinen,Miss
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Futrelle,Mrs
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Allen,Mr


In [0]:
all["Cabin"].fillna("U", inplace=True)
all["Cabin"] = all["Cabin"].apply(lambda x:x[0])

In [176]:
all.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Surname,Title
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Braund,Mr
1,38.0,C,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Cumings,Mrs
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Heikkinen,Miss
3,35.0,C,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Futrelle,Mrs
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Allen,Mr


## Ageの欠損処理について
PclassとTitle（敬称）ごとの平均で補完。
## Fareの欠損処理について
Pclass=3の人なのでPclass=3の平均で補完

In [177]:
Age_means_gro = all.groupby(["Pclass", "Title"], as_index=False)["Age"].mean()
    
Age_mask = all["Age"].map(lambda x: np.isnan(x))
all.loc[Age_mask, "Age"] = all.loc[Age_mask, ["Pclass", "Title"]].map(Age_means_gro)

AttributeError: ignored

ダメだ

In [178]:
Age_means_gro = all.groupby(["Pclass", "Title"], as_index=False)["Age"].mean()
Age_means_gro

,Pclass,Title,Age
0,1,Capt,70.000000
1,1,Col,54.000000
2,1,Don,40.000000
3,1,Dona,39.000000
4,1,Dr,45.600000
5,1,Jonkheer,38.000000
6,1,Lady,48.000000
7,1,Major,48.500000
8,1,Master,6.984000
9,1,Miss,30.338983


ここはOK

In [179]:
Age_mask = all["Age"].map(lambda x: np.isnan(x))
Age_mask

0       False
1       False
2       False
3       False
4       False
5        True
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17       True
18      False
19       True
20      False
21      False
22      False
23      False
24      False
25      False
26       True
27      False
28       True
29       True
        ...  
1279    False
1280    False
1281    False
1282    False
1283    False
1284    False
1285    False
1286    False
1287    False
1288    False
1289    False
1290    False
1291    False
1292    False
1293    False
1294    False
1295    False
1296    False
1297    False
1298    False
1299     True
1300    False
1301     True
1302    False
1303    False
1304     True
1305    False
1306    False
1307     True
1308     True
Name: Age, Length: 1309, dtype: bool

ここもOK

In [180]:
all.loc[Age_mask, "Age"] = all.loc[Age_mask, ["Pclass", "Title"]].map(Age_means_gro)

AttributeError: ignored

ここがダメ。PclassとTitleの2つあるのがいけない→1つにまとめちゃおう

In [181]:
all["col_Pclass_Title"] = all.apply(lambda x:"{}_{}".format(x["Pclass"], x["Title"]), axis=1)

age_mean = all.groupby("col_Pclass_Title")["Age"].mean()

col_mask = all["Age"].map(lambda x: np.isnan(x))

all.loc[col_mask, "Age"] = all.loc[col_mask, "col_Pclass_Title"].map(age_mean)

all.drop(columns="col_Pclass_Title", inplace=True)

all

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Surname,Title
0,22.000000,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Braund,Mr
1,38.000000,C,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Cumings,Mrs
2,26.000000,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Heikkinen,Miss
3,35.000000,C,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Futrelle,Mrs
4,35.000000,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Allen,Mr
5,28.318910,U,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877,Moran,Mr
6,54.000000,E,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463,McCarthy,Mr
7,2.000000,U,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909,Palsson,Master
8,27.000000,U,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742,Johnson,Mrs
9,14.000000,U,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736,Nasser,Mrs


In [182]:
kesson_table(all)

,欠損数,%
Age,1,0.076394
Cabin,0,0.000000
Embarked,2,0.152788
Fare,1,0.076394
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


Ageを一人補完出来ていない。Ms.なのでMs.の平均で埋める。

In [183]:
all.groupby(["Title"], as_index=False)["Age"].mean()

,Title,Age
0,Capt,70.000000
1,Col,54.000000
2,Don,40.000000
3,Dona,39.000000
4,Dr,43.825000
5,Jonkheer,38.000000
6,Lady,48.000000
7,Major,48.500000
8,Master,5.562295
9,Miss,21.001247


In [0]:
all["Age"].fillna(28, inplace=True)

In [185]:
kesson_table(all)

,欠損数,%
Age,0,0.000000
Cabin,0,0.000000
Embarked,2,0.152788
Fare,1,0.076394
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [186]:
all.groupby(["Pclass"], as_index=False)["Fare"].mean()

,Pclass,Fare
0,1,87.508992
1,2,21.179196
2,3,13.302889


In [187]:
all.Fare[1043] = 13.302889

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [188]:
kesson_table(all)

,欠損数,%
Age,0,0.000000
Cabin,0,0.000000
Embarked,2,0.152788
Fare,0,0.000000
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [189]:
all["Embarked"] = all["Embarked"].fillna("S")
all.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Surname,Title
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Braund,Mr
1,38.0,C,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Cumings,Mrs
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Heikkinen,Miss
3,35.0,C,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Futrelle,Mrs
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Allen,Mr


欠損埋め終わり。次はカテゴリ変数化

In [190]:
cols = ["Sex","Embarked","Cabin","Title"]
all_to_cat = pd.get_dummies(all, columns=cols)
print(all_to_cat.shape)

(1309, 42)


In [191]:
data_counts = all['Title'].value_counts()
Title_to_drop = data_counts[data_counts < 3].index.tolist()

to_drop_list = []
target_col = "Title"

for each_data in Title_to_drop:
    col_name = "{}_{}".format(target_col, each_data)
    to_drop_list.append(col_name)
    
all_to_cat.drop(columns=to_drop_list, inplace=True)
all_to_cat.head()

,Age,Fare,Name,Parch,PassengerId,Pclass,SibSp,Survived,Ticket,Surname,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_ Col,Title_ Dr,Title_ Master,Title_ Miss,Title_ Mr,Title_ Mrs,Title_ Rev
0,22.0,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,0.0,A/5 21171,Braund,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,38.0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,1.0,PC 17599,Cumings,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,26.0,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,1.0,STON/O2. 3101282,Heikkinen,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,35.0,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,1.0,113803,Futrelle,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,35.0,8.0500,"Allen, Mr. William Henry",0,5,3,0,0.0,373450,Allen,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [192]:
to_drop = ["PassengerId", "Name", "Ticket", "Surname"]

all_to_cat.drop(columns=to_drop, inplace=True)
print(all_to_cat.shape)

(1309, 27)


In [195]:
all_to_cat["Pclass"] /= 3
all_to_cat["Age"] /= 80
all_to_cat["SibSp"] /= 8
all_to_cat["Parch"] /= 9
all_to_cat["Fare"] /= 512.3292

all_to_cat.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_ Col,Title_ Dr,Title_ Master,Title_ Miss,Title_ Mr,Title_ Mrs,Title_ Rev
0,0.2750,0.014151,0.0,1.000000,0.125,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,0.4750,0.139136,0.0,0.333333,0.125,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0.3250,0.015469,0.0,1.000000,0.000,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,0.4375,0.103644,0.0,0.333333,0.125,1.0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0.4375,0.015713,0.0,1.000000,0.000,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [196]:
train_to_cat = all_to_cat.iloc[0:891, :]
train_to_cat.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_ Col,Title_ Dr,Title_ Master,Title_ Miss,Title_ Mr,Title_ Mrs,Title_ Rev
0,0.2750,0.014151,0.0,1.000000,0.125,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,0.4750,0.139136,0.0,0.333333,0.125,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0.3250,0.015469,0.0,1.000000,0.000,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,0.4375,0.103644,0.0,0.333333,0.125,1.0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0.4375,0.015713,0.0,1.000000,0.000,0.0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 32
num_classes = 2
epochs = 20

In [0]:
from sklearn.model_selection import train_test_split

In [199]:
train_train = train_to_cat.drop(["Survived"], axis=1)
train_train.head()

,Age,Fare,Parch,Pclass,SibSp,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_ Col,Title_ Dr,Title_ Master,Title_ Miss,Title_ Mr,Title_ Mrs,Title_ Rev
0,0.2750,0.014151,0.0,1.000000,0.125,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,0.4750,0.139136,0.0,0.333333,0.125,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0.3250,0.015469,0.0,1.000000,0.000,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
3,0.4375,0.103644,0.0,0.333333,0.125,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0.4375,0.015713,0.0,1.000000,0.000,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [0]:
target = train_to_cat["Survived"].values
features = train_train.values

In [0]:
x_train, x_test, t_train, t_test = train_test_split(features, target, test_size =0.3, random_state = 0)

In [202]:
x_train.shape

(623, 26)

In [0]:
x_train = x_train.reshape(623, 26)
x_test = x_test.reshape(268, 26)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
t_train = ks.utils.to_categorical(t_train, num_classes)
t_test = ks.utils.to_categorical(t_test, num_classes)

In [205]:
model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(26, )))
model.add(Dense(2, activation='softmax'))

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

W0711 09:34:29.148140 139838096283520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 09:34:29.207807 139838096283520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 09:34:29.221302 139838096283520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 09:34:29.278004 139838096283520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0711 09:34:29.288449 139838096283520 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 216       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 234
Trainable params: 234
Non-trainable params: 0
_________________________________________________________________


In [206]:
history = model.fit(x_train, t_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, t_test))

W0711 09:34:55.696827 139838096283520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 623 samples, validate on 268 samples
Epoch 1/20
623/623 [==============================] - 1s 1ms/step - loss: 0.7841 - acc: 0.2857 - val_loss: 0.7064 - val_acc: 0.6231
Epoch 2/20
623/623 [==============================] - 0s 169us/step - loss: 0.6633 - acc: 0.6533 - val_loss: 0.6170 - val_acc: 0.7127
Epoch 3/20
623/623 [==============================] - 0s 180us/step - loss: 0.5938 - acc: 0.7191 - val_loss: 0.5574 - val_acc: 0.7575
Epoch 4/20
623/623 [==============================] - 0s 168us/step - loss: 0.5449 - acc: 0.7560 - val_loss: 0.5094 - val_acc: 0.7836
Epoch 5/20
623/623 [==============================] - 0s 168us/step - loss: 0.5082 - acc: 0.7817 - val_loss: 0.4802 - val_acc: 0.7873
Epoch 6/20
623/623 [==============================] - 0s 162us/step - loss: 0.4918 - acc: 0.7865 - val_loss: 0.4692 - val_acc: 0.7873
Epoch 7/20
623/623 [==============================] - 0s 174us/step - loss: 0.4830 - acc: 0.7897 - val_loss: 0.4618 - val_acc: 0.7873
Epoch 8/20
623/623

In [207]:
test_to_cat = all_to_cat.iloc[891:, :]
test_to_cat.head()

,Age,Fare,Parch,Pclass,SibSp,Survived,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_ Col,Title_ Dr,Title_ Master,Title_ Miss,Title_ Mr,Title_ Mrs,Title_ Rev
891,0.43125,0.015282,0.000000,1.000000,0.000,NaN,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
892,0.58750,0.013663,0.000000,1.000000,0.125,NaN,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
893,0.77500,0.018909,0.000000,0.666667,0.000,NaN,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
894,0.33750,0.016908,0.000000,1.000000,0.000,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
895,0.27500,0.023984,0.111111,1.000000,0.125,NaN,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [208]:
test_test = test_to_cat.drop(["Survived"], axis=1)
test_test.head()

,Age,Fare,Parch,Pclass,SibSp,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Title_ Col,Title_ Dr,Title_ Master,Title_ Miss,Title_ Mr,Title_ Mrs,Title_ Rev
891,0.43125,0.015282,0.000000,1.000000,0.000,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
892,0.58750,0.013663,0.000000,1.000000,0.125,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
893,0.77500,0.018909,0.000000,0.666667,0.000,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
894,0.33750,0.016908,0.000000,1.000000,0.000,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
895,0.27500,0.023984,0.111111,1.000000,0.125,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


In [0]:
test_features = test_test.values

In [210]:
predict_classes = model.predict_classes(test_features, batch_size=batch_size)

predict_classes

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [0]:
PassengerId = np.array(test["PassengerId"]).astype(int)
                            
my_solution = pd.DataFrame(predict_classes, PassengerId, columns = ["Survived"])
                            
my_solution.to_csv("Solution8.csv", index_label = ["PassengerId"])

In [0]:
from google.colab import files
files.download("Solution8.csv")

78.468%。ニューラルネットワークでは最高記録だが過去最高記録タイ。8割超えたい。